# Tarea 1

En esta tarea vas a programar tu mismo el algoritmo de descenso del gradiente, en dos versiones, y comparar ambos resultados. Para probar tu modelo vas a tener que usar el _dataset_ de gustos de helados que vimos en clases.

## Parte 1 (3 pts): gradient descent simple

En el codigo de abajo hay una clase llamada `LogisticRegression` cuyo constructor recibe como parámetro el número de _features_ que espera recibir. Tienes que completar esta clase para que pueda entrenar y predecir. Lo que necesitas es:

- Programar el método `train`, que vendría a ser equivalente al método `fit` de Scikit Learn. Tienes que utilizar el algoritmo _Gradient Descent_ visto en clases.
- Programar el método `predict` que asume que tu modelo ya está entrenado.

Para hacer esto puedes hacer los supuestos razonables que estimes conveniente. Además, si te acomoda trabajar sin clases puedes hacerlo, mientras uses el algoritmo de _Gradient Descent_. **Importante**: puedes asumir que una instancia es "positiva" (es decir, pertenece a la clase 1) si la probabilidad calculada es mayor o igual a 0.5.

Recuerda además que el gradiente de la función objetivo para cada $\beta_i$ es:

$$
\frac{\delta}{\delta \beta_i}L(\beta) = \frac{1}{m} \sum_{1 \leq j \leq m} (\sigma(\beta^T x^j) - y_j) x_i^j
$$

Donde $L(\beta)$ es la función de verosimilitud, $\beta$ es el vector de coeficientes para la regresión, tenemos $m$ filas en nuestro _dataset_, $\sigma(x)$ es la función $\frac{1}{1 + e^{-x}}$, $x^j$ es la fila $j$ de nuestro dataset (y asociado tiene su respuesta $y_j$) y finalmente $x_i^j$ es la columna $i$ de la fila $j$ en nuestro _dataset_.

In [73]:
import numpy as np

# La función sigmoide
def sigmoid(x):    
    output = 1 / (1 + np.exp(-x))
    return output

class LogisticRegression:
    def __init__(self, number_of_features, learning_rate=0.001, number_of_iterations=100):
        self.learning_rate = learning_rate
        self.number_of_iterations = number_of_iterations
        self.beta = np.random.randn(number_of_features, 1)
        
    def train(self, X, y):
        self.X = X 
        self.y = y
        
        for i in range(self.number_of_iterations): 
            self.cambiando_peso() 
        return self
    
    def cambiando_peso(self):
        y_predecido = self.predict(self.X)
        error = y_predecido - self.y.values.reshape(-1, 1)
        dW = (self.X.T).dot(error) / self.X.shape[0]
        self.beta = self.beta - self.learning_rate * dW
    
    def predict(self, value):
        probabilidad = sigmoid(value.dot(self.beta))
        return np.where(probabilidad >= 0.5, 1, 0)

# Ejemplo de uso de la clase para 3 features
log_reg = LogisticRegression(3)
print(log_reg.beta)

[[ 0.20594365]
 [-1.32955507]
 [ 0.56162231]]


## Parte 2 (1 pto): entrenando el modelo, computando el log loss

En esta parte tendrás que hacer un clasificador de gustos de helados tal como lo hicimos para regresión logística. 

In [74]:
import pandas as pd
helados = pd.read_csv('Ice_cream.csv')
helados.info()
X = helados[['female', 'video', 'puzzle']]
y = helados['ice_cream']


# Entrena el modelo acá, debería ser como la siguiente línea
log_reg.train(X, y)

# Ahora usa el predict
predicciones = log_reg.predict(X)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Unnamed: 0  200 non-null    int64
 1   id          200 non-null    int64
 2   female      200 non-null    int64
 3   ice_cream   200 non-null    int64
 4   video       200 non-null    int64
 5   puzzle      200 non-null    int64
dtypes: int64(6)
memory usage: 9.5 KB


In [75]:
from sklearn.metrics import log_loss
#Obtiene el log loss
log_loss(helados['ice_cream'], predicciones)

19.463572830123262

## Parte 3 (1.5 ptos) mini-batch gradient descent

Otra forma de implementar el _gradient descent_ es mediante batches, o pedazos. La idea es la siguiente: 
- Se selecciona un número B de pedazos a usar y un número E de épocas
- Para cada época: 
    - Dividir el set de entrenamiento en B pedazos 
    - Para cada pedazo: 
        - calcular el gradiente usando la fórmula, pero solo para los datos de ese pedazo. 
        - realizar una iteración del gradient descent y actualizar los coeficientes
        
En esta última parte, modifica tu clase LogisticRegression para que el train sea hecho con mini-bath gradient descent. Las épocas y los batches deben ser pasados como parámetros. 

In [76]:
# Tienes que programar la parte 3 aquí
import numpy as np

# La función sigmoide
def sigmoid(x):    
    output = 1 / (1 + np.exp(-x))
    return output

class LogisticRegressionBatches:
    def __init__(self, number_of_features,n_batches,n_epochs, learning_rate=0.001):
        self.learning_rate = learning_rate
        self.n_batches = n_batches
        self.n_epochs = n_epochs
        self.beta = np.random.randn(number_of_features, 1)

    # Basado en la respuesta disponible en: https://stackoverflow.com/questions/45113245/how-to-get-mini-batches-in-pytorch-in-a-clean-and-efficient-way
    def batchify(X, y, batch_size):
        m = X.shape[0]
        batches = []
        for i in range(0, m, batch_size):
            X_batch = X[i:i+batch_size]
            y_batch = y[i:i+batch_size]
            batches.append((X_batch, y_batch))
        return batches
  
    def train(self, X, y):
        # Tienes que programar este método
        batches = batchify(X, y, self.n_batches)
        for epoch in self.n_epochs:
            for X_batch, y_batch in batches:
                self.cambiando_peso(X_batch, y_batch)
    
    def cambiando_peso(self, X, y):
        
        y_predecido = self.predict(X).flatten()
        error = y_predecido - y
        dW = (X.T).dot(error.values.reshape(-1, 1)) / X.shape[0]
        self.beta = self.beta - self.learning_rate * dW
        return self
    
    def predict(self, value):
        # Tienes que programar este método
        pass

## Parte 4 (0.5 pts): número de batches y épocas?

Lo último que debes hacer es encontrar un número de batches y épocas de forma que el modelo que entrenas con esos batches y épocas entregue un log-loss similar al que obtuviste con el método de gradient descent en la parte 1-2. 

In [77]:
#Trabaja acá

LogisticRegressionBatches(3,3,3)

X = helados[['female', 'video', 'puzzle']]
y = helados['ice_cream']


# Entrena el modelo acá, debería ser como la siguiente línea
log_reg.train(X, y)

# Ahora usa el predict
predicciones = log_reg.predict(X)

log_loss(helados['ice_cream'], predicciones)

19.463572830123262

## Detalles académicos

La entrega de este control debe ser un solo archivo **Jupyter Notebook**. **La fecha de entrega es hasta el viernes 30 de Agosto, hasta las 20:00 pm**. La nota se calcula como el número de puntos + un punto base. El archivo se entrega en un cuestionario en canvas. 